# Project setup

[Project repo link](https://github.com/nicolasugrinovic/size_depth_disambiguation?tab=readme-ov-file)


[Paper link](https://www.iri.upc.edu/people/nugrinovic/depthsize/paper.pdf)


## Install python packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Dependencies from requirements.txt & packages needed to install pytorch3d
!pip install trimesh open3d pyransac3d fvcore iopath timm

# pytorch3d
import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html

# Detectron2: source - https://detectron2.readthedocs.io/en/latest/tutorials/install.html
!python -m pip install pyyaml==5.1
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# panoptic api
!pip install 'git+https://github.com/cocodataset/panopticapi.git'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.9/693.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 36.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using c

## Clone project repo and download external tools

In [3]:
# clone project repo
!git clone https://github.com/yunijeong5/size_depth_disambiguation.git

# make checkpoints directory to put model
%mkdir /content/size_depth_disambiguation/external/panoptic_deeplab/tools_d2/checkpoints

# copy model checkpoint from Google Drive
!cp -r "/content/drive/My Drive/674 Project/panoptic_deeplab_H_48_os16_mg124_poly_200k_bs64_crop_640_640_coco_dsconv.pth" /content/size_depth_disambiguation/external/panoptic_deeplab/tools_d2/checkpoints/panoptic_deeplab_H_48_os16_mg124_poly_200k_bs64_crop_640_640_coco_dsconv.pth

# Download Midas weights
%cd /content/size_depth_disambiguation/
%mkdir -p ./weights
%cd weights
!wget https://github.com/intel-isl/DPT/releases/download/1_0/dpt_large-midas-2f21e586.pt

Cloning into 'size_depth_disambiguation'...
remote: Enumerating objects: 203, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 203 (delta 34), reused 195 (delta 26), pack-reused 0
Receiving objects: 100% (203/203), 2.36 MiB | 14.13 MiB/s, done.
Resolving deltas: 100% (34/34), done.
/content/size_depth_disambiguation
/content/size_depth_disambiguation/weights
--2024-05-08 23:02:53--  https://github.com/intel-isl/DPT/releases/download/1_0/dpt_large-midas-2f21e586.pt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/isl-org/DPT/releases/download/1_0/dpt_large-midas-2f21e586.pt [following]
--2024-05-08 23:02:53--  https://github.com/isl-org/DPT/releases/download/1_0/dpt_large-midas-2f21e586.pt
Reusing existing connection to github.com:443.
HTTP request sent, awai

# Demo

Errors to fix to run the cell below
-

In [4]:
%cd /content/size_depth_disambiguation/

# model weights downloaded from: https://drive.google.com/file/d/17bl_n0SUXVktA0x2507dlwmti7s8FIpe/view?usp=sharing

# precompute the data
!python precompute_estimation_data.py --input_path=./input/coco_demo --output_path=./precomputed_data/coco_demo/ --mode=smpl_reproj --dataset=demo --model_type=dpt_large --config-file ./external/panoptic_deeplab/tools_d2/configs/COCO-PanopticSegmentation/panoptic_deeplab_H_48_os16_mg124_poly_200k_bs64_crop_640_640_coco_dsconv.yaml --opts MODEL.WEIGHTS ./external/panoptic_deeplab/tools_d2/checkpoints/panoptic_deeplab_H_48_os16_mg124_poly_200k_bs64_crop_640_640_coco_dsconv.pth

/content/size_depth_disambiguation
initialize
device: cuda
Loading config ./external/panoptic_deeplab/tools_d2/configs/COCO-PanopticSegmentation/panoptic_deeplab_H_48_os16_mg124_poly_200k_bs64_crop_640_640_coco_dsconv.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.
The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std
Number of images to process: 2
Doing folder: 
output folder: ./precomputed_data/coco_demo/
  0% 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
 50% 1/2 [00:06<00:06,  6.31s/it]No joints .json!!!
100% 2/2 [00:07<00:00,  3.58s/it]
finished
Done precomputing data


In [5]:
# run the optimization
!python run_optim_demo.py --model_type=dpt_large --input_path=./input/coco_demo --output_path=./output/coco_demo --input=input/coco_demo/*.jpg --mode=smpl_reproj --plane_scale=1.0 --n_iters=2000 --w_ordinal_loss=0 --w_reg_size=0.0 --config-file ./external/panoptic_deeplab/tools_d2/configs/COCO-PanopticSegmentation/panoptic_deeplab_H_48_os16_mg124_poly_200k_bs64_crop_640_640_coco_dsconv.yaml --opts MODEL.WEIGHTS ./external/panoptic_deeplab/tools_d2/checkpoints/panoptic_deeplab_H_48_os16_mg124_poly_200k_bs64_crop_640_640_coco_dsconv.pth

initialize
imgs todo: 1
Doing folder: coco_demo
mode: smpl_reproj

ordinal weight: 0
number of iters: 2000
output folder: ./results/coco_demo/results_smplreproj_w0_sc1.0_its2000_wsc1.0
  0% 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/pytorch3d/io/obj_io.py:544: UserWarning: No mtl file provided
  warnings.warn("No mtl file provided")
100% 1/1 [00:10<00:00, 10.54s/it]
finished


Generates meshes separately for each person. How to put them in a single obj file? Switch modes? < no, only simple and simple_reproj modes are implemented.

# Test/Eval (can't find ground truth dataset :/)

In [ ]:
# load mudpots-3d-eval
!cp -r  "/content/drive/My Drive/674 Project/mupots-3d-eval/" "/content/size_depth_disambiguation/input/"

In [ ]:
%cd /content/size_depth_disambiguation/input/mupots-3d-eval/
%ls

/content/size_depth_disambiguation/input/mupots-3d-eval
get_mupots-3d.sh  mpii_mupots_config.m            README.txt
license.txt       mpii_mupots_multiperson_eval.m  util/


In [ ]:
# !bash get_mupots-3d.sh
!wget http://gvv.mpi-inf.mpg.de/projects/SingleShotMultiPerson/MultiPersonTestSet.zip

--2024-05-08 02:24:17--  http://gvv.mpi-inf.mpg.de/projects/SingleShotMultiPerson/MultiPersonTestSet.zip
Resolving gvv.mpi-inf.mpg.de (gvv.mpi-inf.mpg.de)... 139.19.206.39
Connecting to gvv.mpi-inf.mpg.de (gvv.mpi-inf.mpg.de)|139.19.206.39|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gvv.mpi-inf.mpg.de/projects/SingleShotMultiPerson/MultiPersonTestSet.zip [following]
--2024-05-08 02:24:18--  https://gvv.mpi-inf.mpg.de/projects/SingleShotMultiPerson/MultiPersonTestSet.zip
Connecting to gvv.mpi-inf.mpg.de (gvv.mpi-inf.mpg.de)|139.19.206.39|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://vcai.mpi-inf.mpg.de/projects/SingleShotMultiPerson/MultiPersonTestSet.zip [following]
--2024-05-08 02:24:18--  https://vcai.mpi-inf.mpg.de/projects/SingleShotMultiPerson/MultiPersonTestSet.zip
Resolving vcai.mpi-inf.mpg.de (vcai.mpi-inf.mpg.de)... 139.19.206.192
Connecting to vcai.mpi-inf.mpg.de (vcai.m

In [ ]:
!unzip MultiPersonTestSet.zip

In [ ]:
!cp -r MultiPersonTestSet "/content/size_depth_disambiguation/input/mupots-3d/"

In [ ]:
%cd /content/size_depth_disambiguation/

/content/size_depth_disambiguation


In [ ]:
!apt install octave

In [ ]:
# How to run octave
!octave -W file.m

In [ ]:
!python eval.py

Traceback (most recent call last):
  File "/content/size_depth_disambiguation/eval.py", line 549, in <module>
    mains()
  File "/content/size_depth_disambiguation/eval.py", line 545, in mains
    pair_depth_order_generic(evals_list=evals_list, eval_name='all_data_depth_order')
  File "/content/size_depth_disambiguation/eval.py", line 514, in pair_depth_order_generic
    dataset = Mupots3D_v2(folder=folder, evaluators=evaluators, with3d=True,
  File "/content/size_depth_disambiguation/eval.py", line 92, in __init__
    with open(f, 'rb') as data:
FileNotFoundError: [Errno 2] No such file or directory: '/home/nugrinovic/datasets/SMPL_MultiP_datasets/mupots-3d/files/eval.pkl'
